In [44]:
file='NR-AR-LBD_wholetraining.smiles.txt'
clean_file = file[:-4] + "_out.txt"
open_file = open(file,"r").readlines()

output = open(clean_file,"w")
finished_list = []
for entry in open_file:
    if entry.split()[0] not in finished_list:
        finished_list.append(entry.split()[0])
        output.write(entry.split()[0] + " " + entry.split()[1] + "\n")

output.close()

In [45]:
#!/usr/bin/env python 
# coding:utf-8

import time, argparse, gc, os

import numpy as np
import pandas as pd

from rdkit import Chem

from feature import *
import SCFPfunctions as Mf
#import SCFPmodel as Mm

# chainer v2
'''import chainer
import chainer.functions as F
import chainer.links as L
from chainer import cuda, Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Reporter, report, report_scope
from chainer import Link, Chain, ChainList, training
from chainer.datasets import tuple_dataset
from chainer.training import extensions'''


xp=np
#xp=cp

print('Making Training  Dataset...')
file = clean_file
print('Loading smiles: ', file)
smi = Chem.SmilesMolSupplier(file,delimiter=' ',titleLine=False)
mols = [mol for mol in smi if mol is not None]

F_list, T_list = [],[]
for mol in mols:
    if len(Chem.MolToSmiles(mol, kekuleSmiles=True, isomericSmiles=True)) > 400: print("too long mol was ignored")
    else:
        F_list.append(mol_to_feature(mol,-1,400))
        T_list.append(mol.GetProp('_Name'))
Mf.random_list(F_list)
Mf.random_list(T_list)

data_t = xp.asarray(T_list, dtype=xp.int32).reshape(-1,1)
data_f = xp.asarray(F_list, dtype=xp.float32).reshape(-1,1,400,lensize)

dataset = (data_f,data_t)
#train_dataset = datasets.TupleDataset(data_f, data_t) 
print(data_t.shape, data_f.shape)

Making Training  Dataset...
Loading smiles:  NR-AR-LBD_wholetraining.smiles_out.txt
(1586, 1) (1586, 1, 400, 42)


In [46]:
x,y = dataset[0].T,dataset[1]
x=np.moveaxis(x,-1,0)
x.shape

(1586, 42, 400, 1)

In [52]:
np.random.seed(4)
x = np.random.permutation(x)
y = np.random.permutation(y)
x_90 = int(x.shape[0]*0.9)
y_90 = int(y.shape[0]*0.9)
print (x_90,y_90,x.shape[0])
x_train,x_test = x[:x_90],x[x_90:]
y_train,y_test = y[:y_90],y[y_90:]
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape )

1427 1427 1586
(1427, 42, 400, 1) (1427, 1) (159, 42, 400, 1) (159, 1)


In [53]:
import tensorflow as tf
from os import path, getcwd, chdir

In [54]:
config = tf.compat.v1.ConfigProto()
sess = tf.compat.v1.Session(config=config)

In [58]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc') is not None and logs.get('acc')>0.90):
      print("\nReached 60% accuracy so cancelling training!")
      self.model.stop_training = True

In [59]:
def train_model(x,y):
    callbacks = myCallback()
    model = tf.keras.models.Sequential([
            # YOUR CODE STARTS HERE
            tf.keras.layers.Conv2D(128,(3,3),activation='relu',input_shape=(42, 400, 1)),
            tf.keras.layers.MaxPool2D((2,2)),
            tf.keras.layers.Conv2D(256,(3,3),activation='relu'),
            tf.keras.layers.MaxPool2D((2,2)),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=20, activation='relu'),
            tf.keras.layers.Dense(units=5, activation='softmax')
            # YOUR CODE ENDS HERE
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    # model fitting
    history = model.fit(
        # YOUR CODE STARTS HERE
        x, y, epochs=20,callbacks=[callbacks]
        # YOUR CODE ENDS HERE
    )
    # model fitting
    return history.epoch, history,model

In [60]:
_, _,model = train_model(x_train,y_train)

Epoch 1/20
45/45 [==============================] - 72s 2s/step - loss: 1.1488 - accuracy: 0.6776
Epoch 2/20
45/45 [==============================] - 78s 2s/step - loss: 1.0308 - accuracy: 0.6910
Epoch 3/20
45/45 [==============================] - 76s 2s/step - loss: 1.0186 - accuracy: 0.6903
Epoch 4/20
45/45 [==============================] - 86s 2s/step - loss: 0.9848 - accuracy: 0.6938
Epoch 5/20
45/45 [==============================] - 78s 2s/step - loss: 0.9368 - accuracy: 0.6952
Epoch 6/20
45/45 [==============================] - 76s 2s/step - loss: 0.8492 - accuracy: 0.7022
Epoch 7/20
45/45 [==============================] - 80s 2s/step - loss: 0.7787 - accuracy: 0.7239
Epoch 8/20
45/45 [==============================] - 81s 2s/step - loss: 0.6509 - accuracy: 0.7596
Epoch 9/20
45/45 [==============================] - 81s 2s/step - loss: 0.5396 - accuracy: 0.7926
Epoch 10/20
45/45 [==============================] - 80s 2s/step - loss: 0.4435 - accuracy: 0.8325
Epoch 11/20
45/45 [

In [27]:
prediction = model.predict(x_test, batch_size=None, verbose=0, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False)

In [28]:
indexes = tf.argmax(prediction, axis=1)

In [29]:
indexes

<tf.Tensor: shape=(19,), dtype=int64, numpy=array([2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2])>

#test data
file='test_set'
clean_file = file[:-4] + "_out.txt"
open_file = open(file,"r").readlines()

output = open(clean_file,"w")
finished_list = []
for entry in open_file:
    if entry.split()[0] not in finished_list:
        finished_list.append(entry.split()[0])
        output.write(entry.split()[0] + " " + entry.split()[1] + "\n")

output.close()

xp=np
#xp=cp

print('Making Training  Dataset...')
file = clean_file
print('Loading smiles: ', file)
smi = Chem.SmilesMolSupplier(file,delimiter=' ',titleLine=False)
mols = [mol for mol in smi if mol is not None]

F_list, T_list = [],[]
for mol in mols:
    if len(Chem.MolToSmiles(mol, kekuleSmiles=True, isomericSmiles=True)) > 400: print("too long mol was ignored")
    else:
        F_list.append(mol_to_feature(mol,-1,400))
        T_list.append(mol.GetProp('_Name'))
Mf.random_list(F_list)
Mf.random_list(T_list)

data_t = xp.asarray(T_list, dtype=xp.int32).reshape(-1,1)
data_f = xp.asarray(F_list, dtype=xp.float32).reshape(-1,1,400,lensize)
print(data_t.shape, data_f.shape)
test_data = (data_f,data_t)
x_test,y_test = test_data[0].T,test_data[1]
x_test=np.moveaxis(x_test,-1,0)
print (x_test.shape,y_test.shape)

In [62]:
results = model.evaluate(x_test, y_test, batch_size=128)

2/2 [==============================] - 0s 219ms/step - loss: 5.0272 - accuracy: 0.4906


In [38]:
results

[12.67577075958252, 0.1428571492433548]